### Scraping Data for Best Pizza Stores.

**Importing Libraries**

In [166]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from splinter import Browser
import time
import json
import re

**Initializing Functions**

In [167]:
# Defining functions to instantiate webdriver, opening browser and closing browser.

def init_browser():
    # Import Splinter and set the chromedriver path
    executable_path = {"executable_path": "chromedriver.exe"}
    browser = Browser("chrome", **executable_path, headless=False)
    return browser

def open_browser(browser, url):
    browser.visit(url)

def close_browser(browser):
    browser.quit()

**Gather html data using Splinter**

In [168]:
# Use splinter to get the html page into an object
url = "https://www.thedailymeal.com/eat/best-pizza-every-state-slideshow"
browser = init_browser()
open_browser(browser, url)
html = browser.html
# Closing the browser
close_browser(browser)

**Scraping the best pizza stores details from the html object using bs4**

In [177]:
# Use bs4 to scrape the elemnets from the html object
state = []
name = []
city = []
# <div class="image-title slide-title"><h2>Alabama: Bottega Café (Birmingham)</h2></div>

soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', {'class':'slide-main'})
print(len(results))

for item in results:
    state_string = item.find('div', {'class' : ['image-title', 'slide-title']}).find('h2').text
    try:
        pattern = re.compile(r'Washington, D.C')
        if re.match(pattern, state_string.split(':')[0]):
            continue
        else:

            #state.append(state_string.split(':')[0])
            state.append(str.strip(state_string.split(':')[0]))
            name.append(str.strip(state_string.split(':')[1].split('(')[0]))
            city.append(str.strip(state_string.split(':')[1].split('(')[1].split(')')[0]))

    except Exception as e:
        print(f"Skipping....")


        
    print(f"{state} {name} {city}")


    
    

51
['Alabama'] ['Bottega Café'] ['Birmingham']
['Alabama', 'Alaska'] ['Bottega Café', 'Moose’s Tooth Pub and Pizzeria'] ['Birmingham', 'Anchorage']
['Alabama', 'Alaska', 'Arizona'] ['Bottega Café', 'Moose’s Tooth Pub and Pizzeria', 'Pizzeria Bianco'] ['Birmingham', 'Anchorage', 'Phoenix']
['Alabama', 'Alaska', 'Arizona', 'Arkansas'] ['Bottega Café', 'Moose’s Tooth Pub and Pizzeria', 'Pizzeria Bianco', 'Wood Stone Craft Pizza & Bar'] ['Birmingham', 'Anchorage', 'Phoenix', 'Fayetteville']
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California'] ['Bottega Café', 'Moose’s Tooth Pub and Pizzeria', 'Pizzeria Bianco', 'Wood Stone Craft Pizza & Bar', 'Tony’s Pizza Napoletana'] ['Birmingham', 'Anchorage', 'Phoenix', 'Fayetteville', 'San Francisco']
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado'] ['Bottega Café', 'Moose’s Tooth Pub and Pizzeria', 'Pizzeria Bianco', 'Wood Stone Craft Pizza & Bar', 'Tony’s Pizza Napoletana', 'Pizzeria Locale'] ['Birmingham', 'Anchorage', '

In [178]:
# Verifying the length of arrays to make sure all data is scraped.
print(len(state))
print(len(city))
print(len(name))


50
50
50


In [179]:
# Creating a dataframe for state wise best pizzas
best_pizza_stores = pd.DataFrame({'restaurant_name' : name,
                                 'city' : city,
                                'state': state })
best_pizza_stores.head()

,restaurant_name,city,state
0,Bottega Café,Birmingham,Alabama
1,Moose’s Tooth Pub and Pizzeria,Anchorage,Alaska
2,Pizzeria Bianco,Phoenix,Arizona
3,Wood Stone Craft Pizza & Bar,Fayetteville,Arkansas
4,Tony’s Pizza Napoletana,San Francisco,California


In [181]:
# Writing the dataframe to a csv
best_pizza_stores.to_csv("./Resources/best_pizza_stores.csv")